In [125]:
def intersect(df1, df2, columns):
    df1 = df1.drop_duplicates(columns)
    df2 = df2.drop_duplicates(columns)
    merged = df1.append(df2)
    
    return merged[merged.duplicated(columns)]

In [126]:
def minus(df2, df1, columns):
    df2 = df2[columns]
    df1 = df1[columns]
    return pd.concat([df2, df1, df1]).drop_duplicates(keep=False)

In [127]:
import pandas as pd

In this notebook we will try to answer to some questions about the challange dataset, the questions are:

- Are Holdings in Challange?
- Do they ask us some interactions never happened? 

Let's start by answering to the first quesiton. We will try to find some touples (CustomerIdx, IsinIdx, BuySell) that are in trades with trade status = 'Holding' but are not in trade with trade status != "Holding". If none of these touplets are present in challange then 'Holdings' are not in challenge.

Let's start by importing the dataframes:

In [128]:
trade = pd.read_csv("../data/Trade.csv")
chall = pd.read_csv("../data/Challenge_20180423.csv")

let's take just the interactions form the starting of 2018

In [129]:
trade = trade[trade["TradeDateKey"] > 20180000]
trade = trade.rename(index=str, columns={"TradeDateKey": "DateKey"})
trade_holding = trade[trade["TradeStatus"] == "Holding"]
trade_interest = trade[trade["TradeStatus"] != "Holding"]





In [130]:
trade_holding.head()

,DateKey,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,CustomerInterest
2212067,20180131,989,20305,Sell,34428.0,NaN,Holding,0.0
2212076,20180131,989,21534,Sell,590192.0,NaN,Holding,0.0
2212122,20180131,989,21535,Buy,7904724.0,NaN,Holding,0.0
2212123,20180228,989,21535,Sell,7904724.0,NaN,Holding,0.0
2212208,20180131,989,23023,Buy,1068175.0,NaN,Holding,0.0


In [146]:
print(trade_holding.shape)
print(trade_interest.shape)

(366703, 8)
(208426, 8)


Let's now filter for the unique values of the tuples CustomerIdx, IsinIdx and BuySell

In [133]:
trade_holding = trade_holding.drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])
trade_interest = trade_interest.drop_duplicates(["CustomerIdx", "IsinIdx", "BuySell"])

In [134]:
print(trade_holding.shape)
print(trade_interest.shape)

(366703, 8)
(208426, 8)


let's now take the holdings not in interest

In [135]:
unique_holdings = minus(trade_holding,trade_interest,["CustomerIdx", "IsinIdx", "BuySell"])

In [136]:
unique_holdings.shape

(329982, 3)

Let's see now how many of them are in challenge

In [139]:
holding_in_chall = intersect(unique_holdings, chall, ["CustomerIdx", "IsinIdx", "BuySell"])

print("the holdings not in interest are {}".format(unique_holdings.shape[0]))
print("the holdings in challange but not in trade_interest are {}".format(holding_in_chall.shape[0]))


the uniques holdings are 329982
the holdings in challange but not in trade_interest are 26318


## So holdings are in challange!

# Second Quesiton

Let's now see how many interactions in challange already happened in the past:

In [145]:
trade = pd.read_csv("../data/Trade.csv")
chall_already_happened = intersect(trade, chall, ["CustomerIdx", "IsinIdx", "BuySell"])
print("{} out of {} interactions in challange already happened".format(chall_already_happened.shape[0],chall.shape[0]))

360140 out of 484758 interactions in challange already happened
